# Random Forest Classifier

By Steven Sison on December 16, 2023

## Description

This document will be used for the preliminary training and evaluation of the random forest classifier. The document includes the necessary processes taken to train the model with the default hyperparameters. This also evaluates the performance of the classifier in terms of accuracy, precision, recall, F1-score, training time, and detection time. Furthermore, this document will only use lexical features and will observe the effect of increasing the number of features used in the model. As this is only for preliminary work, no optimizations, except a simple train-test validation, will be carried out.

### Preliminaries

In [ ]:
import pandas as pd                     # For data transformation
import numpy as numpy                   # For scientific calculations
import seaborn as sns                   # For data visualizations
import matplotlib.pyplot as plt         # For plotting
import plotly.graph_objects as go       # For plotting
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import time
from datetime import datetime
import joblib

dataset = pd.read_csv("final_unbalanced_withLexical.csv")      # Loading the dataset
x_train, x_test, y_train, y_test = train_test_split(dataset.drop(columns=['url_type']), dataset['url_type'], test_size = 0.2, random_state=42)

## Model Training and Evaluation

### Base Model using All Lexical Features Generated

In [ ]:
pipeline = Pipeline([
    ('classifier', RandomForestClassifier())
])

pipeline.fit(x_train, y_train)

In [ ]:
y_pred = pipeline.predict(x_test)
print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=pipeline.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = pipeline.classes_)
disp.plot()
plt.show()

### Evaluating the Effect of Balanced and Unbalanced Datasets

In [ ]:
dataset['url_type'].value_counts()

In [ ]:
# Upsampling

from sklearn.utils import resample

dataset_benign = dataset[(dataset['url_type'] == 0)]
dataset_defacement = dataset[(dataset['url_type'] == 1)]
dataset_phishing = dataset[(dataset['url_type'] == 2)]
dataset_malware = dataset[(dataset['url_type'] == 3)]

dataset_benign_upsampled = resample(dataset_benign,
                                        replace=True,
                                        n_samples = dataset_benign.shape[0],
                                        random_state = 15)

dataset_defacement_upsampled = resample(dataset_defacement,
                                        replace=True,
                                        n_samples = dataset_benign.shape[0],
                                        random_state = 15)

dataset_phishing_upsampled = resample(dataset_phishing,
                                        replace=True,
                                        n_samples = dataset_benign.shape[0],
                                        random_state = 15)

dataset_malware_upsampled = resample(dataset_malware,
                                        replace=True,
                                        n_samples = dataset_benign.shape[0],
                                        random_state = 15)


dataset_upsampled = pd.concat([dataset_benign_upsampled, dataset_defacement_upsampled, dataset_malware_upsampled, dataset_phishing_upsampled])

# dataset_upsampled.info(0)
dataset_upsampled['url_type'].value_counts()

x_up_train, x_up_test, y_up_train, y_up_test = train_test_split(dataset_upsampled.drop(columns=['url_type']), dataset_upsampled['url_type'], test_size = 0.2, random_state=42)

In [ ]:

pipeline_up = Pipeline([
    ('classifier', RandomForestClassifier())
])

pipeline_up.fit(x_up_train, y_up_train)
y_up_pred = pipeline_up.predict(x_up_test)
print(classification_report(y_up_test, y_up_pred))

In [ ]:
cm_up = confusion_matrix(y_up_test, y_up_pred, labels=pipeline_up.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = pipeline_up.classes_)
disp.plot()
plt.show()

### Evaluating the Effect of Adding more Lexical Features

In [ ]:
results = []

for i in range(3):
    pipeline = Pipeline([('classifier', RandomForestClassifier())])

    temp_url_features = x_up_train.iloc[:, 0:(25*(i+1))]
    
    pipeline.fit(temp_url_features, y_up_train)
    
    filename =  'rf_lexical_{}.sav'.format((25*(i+1)))
    joblib.dump(pipeline, filename)

    url_type_predict = pipeline.predict(x_up_test.iloc[:, 0:(25*(i+1))])

    accuracy = accuracy_score(y_up_test, url_type_predict)
    recall = recall_score(y_up_test, url_type_predict, average = 'weighted')
    precision = precision_score(y_up_test, url_type_predict, average = 'weighted', zero_division=1)
    f1 = f1_score(y_up_test, url_type_predict, average = 'weighted')
    results.append(((4*(i+1)), accuracy, recall, precision, f1))

In [ ]:
results = pd.DataFrame(results, columns=['Number of Features', 'Accuracy', 'Recall', 'Precision', 'F1-Score'])
results = results.sort_values(by='Number of Features', ascending=True)
print(results)

Observations:
- Increasing the number of features improves all the class weighted metrics of the model at the cost of a higher training time.

## Model Optimizations

### With Cross Validation (No Hyperparameter Tuning and Using Upsampled Dataset)

In [ ]:
from numpy import mean
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from matplotlib import pyplot

def evaluating_model(cv):

    X, y = dataset_upsampled.drop(columns=['url_type']), dataset_upsampled['url_type']

    model = RandomForestClassifier()
    scores = cross_val_score(model, X,y, scoring = "accuracy", cv = cv, n_jobs=1)

    return mean(scores), scores.min(), scores.max()

# Getting the Ideal Score
'''ideal, _, _ = evaluating_model(LeaveOneOut())
print('Ideal: %.3f' % ideal)'''

folds = range(10,11)

means, mins, maxs = list(), list(), list()

for k in folds:
    # define the test condition
    cv = KFold(n_splits=k, shuffle=True, random_state=1)
    # evaluate k value
    k_mean, k_min, k_max = evaluating_model(cv)
    # report performance
    print('> folds=%d, accuracy=%.3f (%.3f,%.3f)' % (k, k_mean, k_min, k_max))
    # store mean accuracy
    means.append(k_mean)
    # store min and max relative to the mean
    mins.append(k_mean - k_min)
    maxs.append(k_max - k_mean)

# line plot of k mean values with min/max error bars
pyplot.errorbar(folds, means, yerr=[mins, maxs], fmt='o')
# plot the ideal case in a separate color
# pyplot.plot(folds, [ideal for _ in range(len(folds))], color='r')
# show the plot
pyplot.show()


### Hyperparameter Tuning

In [8]:
from bayes_opt import BayesianOptimization

def rf_cl_bo(max_depth, n_estimators, min_samples_leaf, min_samples_split):
    params_rf = {}
    params_rf['max_features'] = 'sqrt'
    params_rf['max_depth'] = round(max_depth)
    params_rf['n_estimators'] = round(n_estimators)
    params_rf['min_samples_leaf'] = round(min_samples_leaf)
    params_rf['min_samples_split'] = round(min_samples_split)
    scores = cross_val_score(RandomForestClassifier(random_state=123, **params_rf),
                             x_up_train, y_up_train, scoring='accuracy', cv=5).mean()
    score = scores.mean()
    return score
# Run Bayesian Optimization
start = time.time()
params_rf ={
    'max_depth':(2, 20),
    'n_estimators':(100, 1500),
    'min_samples_leaf':(1,100),
    'min_samples_split':(2,100) 
}
rf_bo = BayesianOptimization(rf_cl_bo, params_rf, random_state=111)
rf_bo.maximize(init_points=20, n_iter=4)
print('It takes %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.9115    | 13.02     | 17.74     | 44.73     | 1.177e+03 |
| 2         | 0.8584    | 7.316     | 15.77     | 4.203     | 688.3     |
| 3         | 0.8432    | 6.296     | 34.43     | 99.09     | 432.8     |
| 4         | 0.7562    | 3.461     | 67.29     | 62.88     | 484.0     |
| 5         | 0.8918    | 10.39     | 12.72     | 9.248     | 1.361e+03 |
| 6         | 0.9166    | 16.29     | 84.22     | 81.89     | 1.487e+03 |
| 7         | 0.8999    | 12.39     | 81.56     | 43.29     | 138.4     |
| 8         | 0.8921    | 10.17     | 11.43     | 82.09     | 1.077e+03 |
| 9         | 0.9039    | 12.18     | 28.15     | 99.85     | 293.3     |
| 10        | 0.9088    | 13.08     | 49.02     | 41.69     | 1.119e+03 |
| 11        | 0.8711    | 7.811     | 40.65     | 33.26     | 1.426e+03 |
| 12        | 0.9219    | 18.54     | 